# Evaluation


In [1]:
with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

with open("GROQ_KEY.txt", "r") as f:
    groq_token = f.read()

In [2]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


In [3]:
import torch

device = "cpu"
if torch.cuda.is_available():
    print("Cuda available")
    device = torch.device('cuda')

Cuda available


### Preparing the Data for Evaluation

In [4]:
import os 
eval_path = r"Evaluation data\Solved PDFs"
solved_pdfs_path = os.listdir(eval_path)
solved_pdfs_path_pdf_files = [os.path.join(eval_path, pdf) for pdf in solved_pdfs_path if pdf.endswith("pdf")]
solved_pdfs_path_txt_files = [os.path.join(eval_path, txt) for txt in solved_pdfs_path if txt.endswith("txt")]

In [5]:
list_tuple_data = []
i = 0
while i < len(solved_pdfs_path_pdf_files):
    tup = (solved_pdfs_path_pdf_files[i], solved_pdfs_path_txt_files[i])
    list_tuple_data.append(tup)
    i += 1

In [6]:
list_tuple_data

[('Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.pdf',
  'Evaluation data\\Solved PDFs\\4b7076dc7989552fc75046c32d068eee9122adff_DEL13-1223-Annex.txt'),
 ('Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.pdf',
  'Evaluation data\\Solved PDFs\\7a3ef5c116cca7f5328c44ef2c3f227f8c90ef60_Zones_Accélérati.txt'),
 ('Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.pdf',
  'Evaluation data\\Solved PDFs\\a7f4c9d0d7c8f3c9cd67b7c80beec26d9c234382_delibs-22-mai-24.txt'),
 ('Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.pdf',
  'Evaluation data\\Solved PDFs\\ecc6f_07-12-juillet.txt'),
 ('Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.pdf',
  'Evaluation data\\Solved PDFs\\fa224c7892c9c4971dda6423c75c97a04f6e3666_del2024_22-defin.txt')]

### Loading Prompts

In [7]:
import json

with open("prompts_preprocessing.json", "r") as f:
    prompts_for_processing = json.load(f)

table_process_prompt = prompts_for_processing["augment_table_prompt"]
summary_prompt = prompts_for_processing["summary_prompt"]

In [8]:
with open("french_prompts.json", "r") as f:
    prompts_query = json.load(f)

In [9]:
pdf_path = list_tuple_data[0][0]
markdown = list_tuple_data[0][1]

In [11]:
from council_rag.preprocessing import preprocess_markdown_text
from council_rag.data_transformations import process_tables, summarize_clusters
from council_rag.preprocessing.preprocessing import unload_cuda

import time

start = time.time()

paragraphs_list, clusters_dict, model = preprocess_markdown_text(markdown,
                                                        model_id ="Jaume/gemma-2b-embeddings", 
                                                        spacy_model="fr_core_news_sm", 
                                                        n_sents_per_para=8,
                                                        device=device)

: 

In [ ]:
unload_cuda()
processed_tables = process_tables(pdf_path, 
                                table_process_prompt, 
                                groq_token)

In [ ]:
unload_cuda()
clusters_dict = summarize_clusters(clusters_dict, 
                                    summary_prompt, 
                                    groq_token, 
                                    model="gemma2-9b-it", 
                                    token_limit=14000, 
                                    sleep_time=60)

end = time.time()
print((end - start)/60)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from council_rag.rag import prepare_data_for_rag

unload_cuda()
clusters_list, paragraphs_list, all_docs = prepare_data_for_rag(clusters_dict, 
                                                                paragraphs_list, 
                                                                processed_tables, 
                                                                splitter=RecursiveCharacterTextSplitter,
                                                                chunk_size=450,
                                                                chunk_overlap=35,
                                                                length_function=len,
                                                                is_separator_regex=False)

In [12]:
from council_rag.rag import shorten_summary_docs

unload_cuda()
all_docs = shorten_summary_docs(all_docs, groq_token, model)

ImportError: cannot import name 'generate_groq_summary' from 'council_rag.preprocessing' (c:\Users\alber\Desktop\Council-Minutes\Alberto Research\RAGTOR\council_rag\preprocessing\__init__.py)